In [29]:
#from ultralytics import YOLO
#from IPython.display import display, Image
#import cv2
#import numpy as np
#from object_detection import ObjectDetection
#import math
#from sort import *

classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign"]

def dist(P, A, B):
    area = abs ( (A[0] - P[0]) * (B[1] - P[1]) - (A[1] - P[1]) * (B[0] - P[0]) )
    AB = ( (A[0] - B[0]) ** 2 + (A[1] - B[1]) ** 2 ) ** 0.5
    return ( area / AB )
#A가 아래 B가 위
L1_A=(260,450)
L1_B=(420,180) #distance 40

L2_A=(350,450) #id 194
L2_B=(463,200) #id 91 distance 30

L3_A=(385,450) #id 60
L3_B=(505,200) #id 30  distance 30

L4_A=(450,450) #id 105
L4_B=(550,200)  #id 85  distacne 30


R1_A=(550,450)
R1_B=(670,200) #distance 40

R2_A=(630,450) #id 194
R2_B=(724,200) #id 91 distance 30

R3_A=(700,450) #id 60  ####기준점
R3_B=(725,200) #id 30  distance 30 

R4_A=(770,450) #id 105
R4_B=(825,180)  #id 85  distacne 30

# Initialize Object Detection
# od = ObjectDetection()
model = YOLO('yolov8n.pt')
cap = cv2.VideoCapture("Tokyo1.mp4")

frameWidth = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))	# 영상의 넓이(가로) 프레임
frameHeight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))	# 영상의 높이(세로) 프레임
frame_size = (frameWidth, frameHeight)

print(frame_size)
# Initialize count
count = 0
center_points_prev_frame = []

danger_car_pt={}
danger_car_id=[]
tracking_objects = {}
track_id = 0
tracker = Sort(max_age=12, min_hits=2, iou_threshold=0.4)

mask = cv2.imread("mmask.png")
mask=cv2.resize(mask,(frameWidth,frameHeight))

while True:
    tracking_objects = {}
    retval, frame = cap.read()
    imgRegion = cv2.bitwise_and(frame,mask)
    count += 1
    if not(retval):
        break
#그리드 그리기

    rows,cols, _ = frame.shape
    cell_size =50
    
    for i in range(0,rows,cell_size):
        cv2.line(frame,(0,i),(cols,i),(255,0,0),1)
    for j in range(0,cols,cell_size):
        cv2.line(frame,(j,0),(j,rows),(255,0,0),1)
#여기까지    
    # Point current frame
    center_points_cur_frame = []

    # Detect objects on frame
    results = model(imgRegion,stream=True)
    detections = np.empty((0,5))
    
    for r in results:
        boxes = r.boxes
        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            cx = int((x1+x2) / 2)
            cy = int((y1+y2) / 2)
            center_points_cur_frame.append((cx, cy, 'Danger!'))
            conf = math.ceil((box.conf[0]*100)) / 100
            cls = int(box.cls[0])
            currentClass = classNames[cls]

            if currentClass == "car" or currentClass == "truck" or currentClass == "bus" \
                    or currentClass == "motorbike" and conf > 0.3:
                cv2.rectangle(frame, (x1, y1), (x2,y2), (0, 255, 0), 2) 
                currentArray = np.array([x1,y1,x2,y2,conf])
                detections = np.vstack((detections,currentArray))

    # Only at the beginning we compare previous and current frame
    
    resultsTracker=tracker.update(detections)
    
    for result in resultsTracker:
        x1,y1,x2,y2,Id = result
        cx = int((x1+x2) / 2)
        cy = int((y1+y2) / 2)
        Id = int(Id)
                
    # for object_id, pt in tracking_objects.items():
        if dist((cx,cy), L1_A, L1_B) < 20:
            tracking_objects[Id]=(cx,cy,'L1')
        
        elif dist((cx,cy), L2_A, L2_B) < 20:
            tracking_objects[Id]=(cx,cy,'L2')
            
        elif dist((cx,cy), L3_A, L3_B) < 20:
            tracking_objects[Id]=(cx,cy,'L3')
        
        elif dist((cx,cy), L4_A, L4_B) < 20:
            tracking_objects[Id]=(cx,cy,'L4')
            
            
        elif dist((cx,cy), R1_A, R1_B) < 20:
            tracking_objects[Id]=(cx,cy,'R1')
        
        elif dist((cx,cy), R2_A, R2_B) < 20:
            tracking_objects[Id]=(cx,cy,'R2')
            
        elif dist((cx,cy), R3_A, R3_B) < 20:
            tracking_objects[Id]=(cx,cy,'R3')
        
        elif dist((cx,cy), R4_A, R4_B) < 20:
            tracking_objects[Id]=(cx,cy,'R4')
        
        else:
            tracking_objects[Id]=(cx,cy,'Danger')
            if Id in danger_car_pt:
                danger_car_pt[Id] +=1
                if danger_car_pt[Id] > 30 and id not in danger_car_id:
                    danger_car_id.append(Id)
                    
            else:
                danger_car_pt[Id] = 1

    for object_id, pt in tracking_objects.items():
        tmp= pt[2]+'['+str(object_id)+']'
        cv2.circle(frame, (pt[0],pt[1]), 5, (0, 0, 255), -1)
        cv2.putText(frame, tmp, (pt[0], pt[1] - 7), 0, 0.8, (0, 0, 255), 2)
    
    cv2.imshow("Frame", frame)

    center_points_prev_frame = [center_points_cur_frame.copy()]
    print(danger_car_id)
    print(danger_car_pt)
    key = cv2.waitKey(0)
    if key == 27:
        break



cap.release()
cv2.destroyAllWindows()

(920, 488)



0: 352x640 1 person, 8 cars, 2 buss, 67.1ms
Speed: 2.5ms preprocess, 67.1ms inference, 1.0ms postprocess per image at shape (1, 3, 352, 640)


[]
{673: 1, 668: 1}



0: 352x640 9 cars, 2 buss, 64.4ms
Speed: 2.5ms preprocess, 64.4ms inference, 1.0ms postprocess per image at shape (1, 3, 352, 640)


[]
{673: 2, 668: 2}


...

[678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 714, 692, 678, 714, 692, 678, 714, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 714, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 714, 692, 678, 692, 678]
{673: 21, 668: 18, 678: 106, 679: 3, 682: 12, 687: 11, 689: 2, 690: 3, 674: 2, 692: 77, 694: 2, 667: 28, 676: 4, 703: 2, 714: 35, 713: 3, 717: 5, 719: 20, 723: 3, 729: 2, 733: 1, 731: 1}
[678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 692, 678, 692, 678, 692, 6

0: 352x640 13 cars, 1 bus, 2 trucks, 64.1ms
Speed: 1.0ms preprocess, 64.1ms inference, 1.0ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 13 cars, 1 bus, 2 trucks, 60.5ms
Speed: 1.0ms preprocess, 60.5ms inference, 1.0ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 14 cars, 1 bus, 2 trucks, 66.2ms
Speed: 2.2ms preprocess, 66.2ms inference, 1.0ms postprocess per image at shape (1, 3, 352, 640)



[678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 714, 692, 678, 714, 692, 678, 714, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 714, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 714, 692, 678, 692, 678, 692, 692, 714, 692, 678]
{673: 21, 668: 18, 678: 107, 679: 3, 682: 12, 687: 11, 689: 2, 690: 3, 674: 2, 692: 80, 694: 2, 667: 28, 676: 4, 703: 2, 714: 36, 713: 3, 717: 5, 719: 23, 723: 3, 729: 2, 733: 3, 731: 2}
[678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 6

0: 352x640 13 cars, 1 bus, 2 trucks, 61.6ms
Speed: 1.3ms preprocess, 61.6ms inference, 1.0ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 11 cars, 1 bus, 2 trucks, 63.9ms
Speed: 2.2ms preprocess, 63.9ms inference, 1.0ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 11 cars, 1 bus, 2 trucks, 67.5ms
Speed: 2.1ms preprocess, 67.5ms inference, 1.0ms postprocess per image at shape (1, 3, 352, 640)


[678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 714, 692, 678, 714, 692, 678, 714, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 714, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 692, 678, 714, 692, 678, 692, 678, 692, 692, 714, 692, 678, 692, 678, 692, 678, 714, 692, 678]
{673: 21, 668: 18, 678: 110, 679: 3, 682: 12, 687: 11, 689: 2, 690: 3, 674: 2, 692: 83, 694: 2, 667: 28, 676: 4, 703: 2, 714: 37, 713: 3, 717: 5, 719: 26, 723: 3, 729: 2, 733: 6, 731: 5, 739: 1}
[678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678, 678

In [5]:
from ultralytics import YOLO
from IPython.display import display, Image
import cv2
import numpy as np
# from object_detection import ObjectDetection
import math
# from sort import *

In [3]:
from sort import *
 

In [11]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

# 이미지 파일 경로
image_path = 'mask.png'

# 이미지 열기
img = Image.open(image_path)

# 이미지 플로팅
plt.imshow(img)

# 점 좌표 (예시)
point_x = 734
point_y = 337

pooint_x = 130
pooint_y = 150

# 점 그리기
plt.scatter(point_x, point_y, c='red', marker='o', s=50)  # 점 색상, 모양, 크기 등 설정
plt.scatter(pooint_x, pooint_y, c='blue', marker='o', s=50)  # 점 색상, 모양, 크기 등 설정

# 플로팅 업데이트
plt.show()


In [37]:
from ultralytics import YOLO
from IPython.display import display, Image
import cv2
import numpy as np
from object_detection import ObjectDetection
import math
from sort import *

classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
              "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush"
              ]

def dist(P, A, B):
    area = abs ( (A[0] - P[0]) * (B[1] - P[1]) - (A[1] - P[1]) * (B[0] - P[0]) )
    AB = ( (A[0] - B[0]) ** 2 + (A[1] - B[1]) ** 2 ) ** 0.5
    return ( area / AB )

f1_A=(388,821)
f1_B=(661,404) #distance 40

f2_A=(582, 876) #id 194
f2_B=(703, 415) #id 91 distance 30

f3_A=(782, 865) #id 60
f3_B=(743, 417) #id 30  distance 30

f4_A=(999, 866) #id 105
f4_B=(789, 417)  #id 85  distacne 30

f5_A=(1183, 850) #id 4
f5_B=(846, 429) #id 29 distance 30

# Initialize Object Detection
# od = ObjectDetection()
model = YOLO('yolov8n.pt')
cap = cv2.VideoCapture("2.mp4")

frameWidth = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))	# 영상의 넓이(가로) 프레임
frameHeight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))	# 영상의 높이(세로) 프레임
frame_size = (frameWidth, frameHeight)

print(frame_size)
# Initialize count
count = 0
center_points_prev_frame = []

danger_car_id=[]

tracking_objects = {}
track_id = 0

tracker = Sort(max_age=10, min_hits=3, iou_threshold=0.3)
mask = cv2.imread("mask.png")

while True:
    retval, frame = cap.read()
    # imgRegion = cv2.bitwise_and(frame,mask)
    count += 1
    if not(retval):
        break

    # Point current frame
    center_points_cur_frame = []

    # Detect objects on frame
    results = model(frame,stream=True)
    detections = np.empty((0,5))
    
    for r in results:
        boxes = r.boxes
        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            cx = int((x1+x2) / 2)
            cy = int((y1+y2) / 2)
            center_points_cur_frame.append((cx, cy, 'Danger!'))
            conf = math.ceil((box.conf[0]*100)) / 100
            cls = int(box.cls[0])
            currentClass = classNames[cls]
            #print("FRAME N°", count, " ", x, y, w, h)
            # cv2.circle(frame, (cx, cy), 5, (0, 0, 255), -1)
            if currentClass == "car" or currentClass == "truck" or currentClass == "bus" \
                    or currentClass == "motorbike" or currentClass == "person" and conf > 0.3:
                cv2.rectangle(frame, (x1, y1), (x2,y2), (0, 255, 0), 2) 
                currentArray = np.array([x1,y1,x2,y2,conf])
                detections = np.vstack((detections,currentArray))

    # Only at the beginning we compare previous and current frame
    

    resultsTracker=tracker.update(detections)
    
    for result in resultsTracker:
        x1,y1,x2,y2,Id = result
        cx = int((x1+x2) / 2)
        cy = int((y1+y2) / 2)
        Id = int(Id)
        # print(cx,cy,Id)
                
    # for object_id, pt in tracking_objects.items():
        # if dist((cx,cy), f1_A, f1_B) < 30:
        #     tracking_objects[Id]=(cx,cy,'f1')
        
        # elif dist((cx,cy), f2_A, f2_B) < 30:
        #     tracking_objects[Id]=(cx,cy,'f2')
            
        # elif dist((cx,cy), f3_A, f3_B) < 30:
        #     tracking_objects[Id]=(cx,cy,'f3')
        
        # elif dist((cx,cy), f4_A, f4_B) < 30:
        #     tracking_objects[Id]=(cx,cy,'f4')
        
        # elif dist((cx,cy), f5_A, f5_B) < 30:
        #     tracking_objects[Id]=(cx,cy,'f5')
        
        # else:
        #     tracking_objects[Id]=(cx,cy,'Danger!')
        
            
            
    # for object_id, pt in tracking_objects.items():
    #     tmp= pt[2]+'['+str(object_id)+']'
    #     cv2.circle(frame, (pt[0],pt[1]), 5, (0, 0, 255), -1)
    #     cv2.putText(frame, tmp, (pt[0], pt[1] - 7), 0, 0.8, (0, 0, 255), 2)
        
    #     if (pt[2]=='Danger!'):
    #         danger_car_id.append(object_id)
            
                
    
    

    # print("Tracking objects")
    # # print(tracking_objects)


    # print("CUR FRAME LEFT PTS")
    # print(center_points_cur_frame)
    
#     cv2.circle(frame, (1,1), 5, (0, 0, 255), -1)
    cv2.imshow("Frame", frame)
    # cv2.imshow("ImageFrame",imgRegion)
    # Make a copy of the points
    center_points_prev_frame = center_points_cur_frame.copy()

    key = cv2.waitKey(0)
    if key == 27:
        break



cap.release()
cv2.destroyAllWindows()


(0, 0)
